<h1 style="text-align:center">Cultural Sites in Italy</h1>
Welcome in the documentation of the information visualization's project Cultural Sites in Italy. The project aims at exploring the data collected in data.beniculturali.it, related to cultural properties, collected by Arco, and cultural events, collected by the MIBACT (today MIC).<br>
The notebook will be devided in two sections according to this difference: the first part will be dedicated to events, while the second to properties.

Before starting, let's import all the libraries necessary and assign the endpoint variables that we will use later:

In [ ]:
from pandas import *

from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.io import output_notebook, show
from bokeh.models import LogColorMapper, ColumnDataSource
from bokeh.palettes import Oranges256 as oranges

from pyproj import Proj, transform

from sparql_dataframe import get
endpoint = 'https://dati.cultura.gov.it/sparql'

endpointDB = 'https://dbpedia.org/sparql'

<h2 style="text-align:center">Cultural Events</h2>
The **CIS (Cultural Institute/Site and Cultural Event) ontology** aims at modelling the data on cultural institutes or sites such as data regarding the agents that play a specific role on cultural institutes or sites, the sites themselves, the contact points, all multimedia files which describe the cultural institute or site and any other information useful to the public in order to access the institute or site.

We specificially employed it to extract all the cultural events present in the dataset through the property **cis:CulturalEvent**.

We get also the cultural events' names, through using the property, **rdfs:label**. We kept the IRI in order to have a unique identifier both for institutes and sites that will help us to merge our tables.

In [ ]:
query_events = """
SELECT DISTINCT ?s ?event WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
}
"""
df = sparql_dataframe.get(endpoint, query_events)

We then employed the property **cis:isHostedBySite** to extract all the sites in which the events were held as this specific property links the Event to the Site of the Cultural Institute or Site.

Also in this case we actually extracted both the IRIs and the label through the same property as the one mentioned above.

In [ ]:
query_site = """
SELECT DISTINCT ?s ?event ?o ?site  WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
}
"""
df = sparql_dataframe.get(endpoint, query_site)

Then we examined various ways to retrieve the city of each city, examing the various predicates associated to them; the most appropriate and helpful way for us was to first extract the address through **cis:siteAddress** and then from here to get the city through the employment of another ontology (Address (Location) Ontology) and its property **clavpit:hasCity**.

In [ ]:
query_culture_events = """
SELECT DISTINCT ?s ?event ?o ?site ?urlcity ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""
df = sparql_dataframe.get(endpoint, query_culture_events)

To further clear up this file from any duplicates that might appear we used the pandas method **drop_duplicates** in order to obtain a clearer table of all the data that we obtained together.

In [ ]:
df=df.drop_duplicates(["s", "o"])

Then we worked onto counting first the events per sites

In [ ]:
query_site = """
SELECT DISTINCT (count (?o) as ?count) ?site WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""
df_count_sites = sparql_dataframe.get(endpoint, query_site)

In [ ]:
query_city = """
SELECT DISTINCT (count (?s) as ?count) ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

df_city = sparql_dataframe.get(endpoint, query_city)

After having extracted all the geographical information, we then moved onto working into the temporal one.

We did this through the employment of the **Time ontology** (Italian application profile) in the properties tiapit:atTime.

This actually gave us a time interval composed by a **start time** and **end time**, which were then explicited through the xsd format with the properties **tiapit:StartMonth** and **tiapit:EndMonth**.

In [ ]:
query_time="""
SELECT DISTINCT ?s ?time ?starttime ?endtime WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s tiapit:atTime ?time.
?time tiapit:startTime ?startTime;
tiapit:endTime ?endtime.
}"""
df_time = sparql_dataframe.get(endpoint, query_time)

Similarly as we had done with the geographical information we counted how many events through the years.

In [ ]:
query_count_time ="""
SELECT DISTINCT (count (?s) as ?count)  (year(xsd:dateTime(?starttime)) as ?StartYear) (year(xsd:dateTime(?endtime)) as ?EndYear) WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s tiapit:atTime ?time.
?time tiapit:startTime ?starttime;
tiapit:endTime ?entime.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}"""

df_time_count = sparql_dataframe.get(endpoint, query_count_time)

<h2 style="text-align:center">Cultural Events</h2>
Arco ontology tries to give the widest representation of italian cultural heritage, inglobing all the main classifications provided by international organizations. The class <b>arco:CulturalProperty</b> is the higher class, however it is not always assigned to all the cultural properties in the dataset. Furthermore each cultural property can belong to different classes. For this reason, in order to retrieve all the cultural properties, we applyed a filter which lists all the possible classes and subclasses that have as type on of the cultural properties'classes.

In [ ]:
query = '''
SELECT DISTINCT(?o) WHERE {
?s a ?o.
FILTER (?o = arco:CulturalProperty || ?o = arco:IntangibleCulturalProperty || ?o = arco:TangibleCulturalProperty || ?o = arco:ArchaeologicalProperty || ?o = arco:ImmovableCulturalProperty || ?o = arco:ArchitecturalOrLandscapeHeritage || ?o = arco:HistoricOrArtisticProperty || ?o = arco:MusicHeritage || ?o = arco:NaturalHeritage || ?o = arco:BotanicalHeritage || ?o = arco:MineralHeritage || ?o = arco:PalaeontologicalHeritage || ?o = arco:PertologicHeritage || ?o = arco:PlanetaryScienceHeritage || ?o = arco:ZoologicalHeritage || ?o = arco:NumismaticProperty || ?o = arco:PhotographicHeritage || ?o = arco:ScientificOrTechnologicalHeritage)
}
'''

CulturalProperties = get(endpoint, query)
CulturalProperties

In [ ]:
query = '''
SELECT DISTINCT(?institute) COUNT(DISTINCT(?s) as ?count) ?instituteLabel WHERE {
?s a ?o.
FILTER (?o = arco:CulturalProperty || ?o = arco:IntangibleCulturalProperty || ?o = arco:TangibleCulturalProperty || ?o = arco:ArchaeologicalProperty || ?o = arco:ImmovableCulturalProperty || ?o = arco:ArchitecturalOrLandscapeHeritage || ?o = arco:HistoricOrArtisticProperty || ?o = arco:MusicHeritage || ?o = arco:NaturalHeritage || ?o = arco:BotanicalHeritage || ?o = arco:MineralHeritage || ?o = arco:PalaeontologicalHeritage || ?o = arco:PertologicHeritage || ?o = arco:PlanetaryScienceHeritage || ?o = arco:ZoologicalHeritage || ?o = arco:NumismaticProperty || ?o = arco:PhotographicHeritage || ?o = arco:ScientificOrTechnologicalHeritage)
?s a-loc:hasCulturalInstituteOrSite ?institute.
?institute rdfs:label ?instituteLabel.
}
'''

CulturalInstitutes = get(endpoint, query)
CulturalInstitutes